This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***

Abstract:  The main goal of the paper is to extract Morbidity from clinical notes.  The idea was to use a combination of classical and deep learning methods to determine the best approach for classifying these notes in one or more of 16 morbidity conditions.  These models used a combination of NLP techniques including embeddings and bag of words implementations.  It also measured the effect including of stop words.  Lastly, it used ensemble techniques to tie together a number of the classical and deep learning models to provide the most accurate results.

Dataset was retrieved from the DBMI Data Portal, Department of Biomedical Informatics (DBMI) in the Blavatnik Institute at Harvard Medical School.  This dataset was originally created for the i2b2 Obesity Challenge conducted in 2008.
This data was provided in XML format with a test and training set.  Along with the test and training set, labeled data of two forms were included. They were called Intuitive and Textual.  Textual judgements were derived by looking at the notes by multiple experts.  When the experts didn’t agree, a resident doctor annotated it with a Intuitive judgement.

In this workbook, we are taking the following steps:


* Loading test and train data along with annotations
* Exploring the best annotation data sets to use
* Preprocessing the data using NLP techniques described below.
* Saving the data as pkl files for use in additional notebooks.





 

In [1]:
pip install xmltodict


Note: you may need to restart the kernel to use updated packages.


The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [2]:
DATA_PATH = './obesity_data/'

Next we create a function to load the data from XML files and convert to a more usable dataframe structure.

In [3]:
import pandas as pd
import xmltodict

def load_dataset(filepath, xpath):    
    return pd.read_xml(filepath, xpath=xpath)

def load_annotations(filepath):

  with open(filepath,"r") as f:
      data = f.read()

  df = pd.DataFrame(columns=['source','disease','id','judgment'])

  data = xmltodict.parse(data)['diseaseset']['diseases']

  for key,val in enumerate(data):
    if(isinstance(val,str)):
      source = data['@source']
      disease = data['disease']
    else:
      source = val['@source']
      disease = val['disease']

    for key,val in enumerate(disease):
      if(isinstance(val,str)):
        disease_name = disease['@name']
        doc = disease['doc']
      else:
        disease_name = val['@name']
        doc = val['doc']
      
      for key,val in enumerate(doc):
        if(isinstance(val,str)):
          doc_id = doc['@id']
          judgment = doc['@judgment']
        else:
          doc_id = val['@id']
          judgment = val['@judgment']
        df_temp = pd.DataFrame([{"source":source,"disease":disease_name,"id":doc_id,"judgment":judgment}])
        #df = df.append(df_temp)  
        df = pd.concat([df,df_temp])

  #The xml acts really strange if there are single nodes.  Dropping duplicates solves it.
  return df.drop_duplicates()

Now we load the test and train datasets and examine the notes. Note, we are loading the training file with 2 as a seperate data frame as it relates to all the addendums which we believe was not used by the paper.

In [4]:
test_df = load_dataset(DATA_PATH + 'obesity_patient_records_test.xml', xpath='/root/docs/*')
test_df['id'] = pd.to_numeric(test_df['id'])
print(test_df.head())
print(len(test_df))

train_df = load_dataset(DATA_PATH + 'obesity_patient_records_training.xml', xpath='/root/docs/*')
train_df_with2 = train_df.append(load_dataset(DATA_PATH + '/obesity_patient_records_training2.xml', xpath='/root/docs/*'))
train_df['id'] = pd.to_numeric(train_df['id'])
train_df_with2['id'] = pd.to_numeric(train_df_with2['id'])
print(train_df.head())
print(len(train_df))
print(len(train_df_with2))

print(test_df['text'][0])

   id                                               text
0   3  470971328 | AECH | 09071283 | | 6159055 | 5/26...
1   5  508283935 | KFM | 67491508 | | 9707967 | 9/25/...
2   7  248652055 | CM | 07563073 | | 5027467 | 8/29/2...
3   8  052907410 | FTH | 50999409 | | 7815179 | 10/6/...
4   9  628477951 | MBCH | 30737210 | | 5713924 | 12/1...
507
   id                                               text
0   1  490646815 | WMC | 31530471 | | 9629480 | 11/23...
1   2  159644670 | VH | 60656526 | | 6334749 | 11/29/...
2   4  368346277 | EMH | 64927307 | | 815098 | 3/29/1...
3   6  018858680 | AOH | 80239131 | | 9725704 | 11/4/...
4  13  908761918 | MMC | 45427009 | | 0927689 | 5/26/...
611
730
470971328 | AECH | 09071283 | | 6159055 | 5/26/2006 12:00:00 AM | PNUEMONIA | Signed | DIS | Admission Date: 4/22/2006 Report Status: Signed

Discharge Date: 7/27/2006
ATTENDING: CARINE , WALTER MD
SERVICE:
Medicine Service.
ADMISSION INFORMATION AND CHIEF COMPLAINT:
Hypoxemic respiratory failure.
HISTO

C:\Users\mclop\AppData\Local\Temp\ipykernel_3268\201322925.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_with2 = train_df.append(load_dataset(DATA_PATH + '/obesity_patient_records_training2.xml', xpath='/root/docs/*'))


The annotation data came in two forms: textual and intuitive.  It also came with files with the forms in seperate files and with the forms all together in one file.  We do some exploration to determine which set of data is the closest to the study.

In [5]:
test_annot_intuitive_df = load_annotations(DATA_PATH + "obesity_standoff_annotations_test_intuitive.xml")
test_annot_intuitive_df['id'] = pd.to_numeric(test_annot_intuitive_df['id'])

test_annot_textual_df = load_annotations(DATA_PATH + "obesity_standoff_annotations_test_textual.xml")
test_annot_textual_df['id'] = pd.to_numeric(test_annot_textual_df['id'])

print(test_annot_intuitive_df.head())
print(len(test_annot_intuitive_df))

print(test_annot_textual_df.head())
print(len(test_annot_textual_df))


      source disease  id judgment
0  intuitive  Asthma   3        Y
0  intuitive  Asthma   5        N
0  intuitive  Asthma   7        N
0  intuitive  Asthma   9        Y
0  intuitive  Asthma  10        N
7399
    source disease  id judgment
0  textual  Asthma   3        Y
0  textual  Asthma   5        U
0  textual  Asthma   7        U
0  textual  Asthma   8        U
0  textual  Asthma   9        Y
8044


The test file with all forms is explored and the record count seems the same as combining the seperate files.

In [6]:
#trying to verify the same number of records in the one with both intuitive and textual
test_annot_all_df = load_annotations(DATA_PATH + "obesity_standoff_annotations_test.xml")
test_annot_all_df['id'] = pd.to_numeric(test_annot_all_df['id'])

print(test_annot_all_df.head())
print(len(test_annot_all_df))

      source disease  id judgment
0  intuitive  Asthma   3        Y
0  intuitive  Asthma   5        N
0  intuitive  Asthma   7        N
0  intuitive  Asthma   9        Y
0  intuitive  Asthma  10        N
15443


We then do the same analysis with the training annotations.

In [7]:
train_annot_intuitive_df = load_annotations(DATA_PATH + "obesity_standoff_intuitive_annotations_training.xml")
train_annot_intuitive_df['id'] = pd.to_numeric(train_annot_intuitive_df['id'])
train_annot_textual_df = load_annotations(DATA_PATH + "obesity_standoff_textual_annotations_training.xml")
train_annot_textual_df['id'] = pd.to_numeric(train_annot_textual_df['id'])

print(train_annot_intuitive_df.head())
print(len(train_annot_intuitive_df))

print(train_annot_textual_df.head())
print(len(train_annot_textual_df))

      source disease  id judgment
0  intuitive  Asthma   1        N
0  intuitive  Asthma   2        Y
0  intuitive  Asthma   4        N
0  intuitive  Asthma   6        N
0  intuitive  Asthma  15        N
8621
    source disease  id judgment
0  textual  Asthma   1        U
0  textual  Asthma   2        Y
0  textual  Asthma   4        U
0  textual  Asthma   6        U
0  textual  Asthma  13        U
9655


When we look at the full file with addendums, we see there is a lot more data in the full file than the seperate file.

In [8]:
#trying to verify the same number of records in the one with both intuitive and textual (It isn't according to tally.pdf it should be 22285 with the annotations and addendums)
train_annot_all_df = load_annotations(DATA_PATH + "obesity_standoff_annotations_training.xml")
train_annot_all_df_with2 = pd.concat([train_annot_all_df,load_annotations(DATA_PATH + "obesity_standoff_annotations_training_addendum.xml")])
train_annot_all_df_with2 = pd.concat([train_annot_all_df_with2,load_annotations(DATA_PATH + "obesity_standoff_annotations_training_addendum2.xml")])
train_annot_all_df_with2 = pd.concat([train_annot_all_df_with2,load_annotations(DATA_PATH + "obesity_standoff_annotations_training_addendum3.xml")])

train_annot_all_df['id'] = pd.to_numeric(train_annot_all_df['id'])
train_annot_all_df_with2['id'] = pd.to_numeric(train_annot_all_df_with2['id'])

print(train_annot_all_df.head())
print(len(train_annot_all_df))
print(len(train_annot_all_df_with2))


      source disease  id judgment
0  intuitive  Asthma   1        N
0  intuitive  Asthma   2        Y
0  intuitive  Asthma   4        N
0  intuitive  Asthma   6        N
0  intuitive  Asthma  15        N
18276
22285


We are going to start with the annotations in one file (test_annot_all_df, train_annot_all_df).  The paper only used annotations that were clearly marked 'Y' or 'N' (It excluded the 'Q' and 'U').

In [9]:
print(len(test_annot_all_df),len(train_annot_all_df))
test_annot_all_df_clean = test_annot_all_df[(test_annot_all_df['judgment']  == 'Y') | (test_annot_all_df['judgment']  == 'N')]
train_annot_all_df_clean = train_annot_all_df[(train_annot_all_df['judgment']  == 'Y') | (train_annot_all_df['judgment']  == 'N')]
print(len(test_annot_all_df_clean),len(train_annot_all_df_clean))


15443 18276
9642 11274


In [10]:
print(test_annot_all_df_clean.groupby('disease').size())


disease
Asthma                  541
CAD                     756
CHF                     650
Depression              549
Diabetes                829
GERD                    494
Gallstones              580
Gout                    552
Hypercholesterolemia    650
Hypertension            826
Hypertriglyceridemia    496
OA                      544
OSA                     562
Obesity                 648
PVD                     528
Venous Insufficiency    437
dtype: int64


In [11]:

print(train_annot_all_df_clean.groupby('disease').size())


disease
Asthma                  648
CAD                     897
CHF                     489
Depression              668
Diabetes                978
GERD                    586
Gallstones              687
Gout                    667
Hypercholesterolemia    757
Hypertension            983
Hypertriglyceridemia    602
OA                      654
OSA                     678
Obesity                 801
PVD                     639
Venous Insufficiency    540
dtype: int64


The paper specifically calls out 6 files and does not mention the addendums, so we will stick with the seperately labeled files for our study. There seems to be only one record in each of the test and training set where the textual and intuitive disagree.

In [12]:
test_annot_intuitive_df_clean = test_annot_intuitive_df[(test_annot_intuitive_df['judgment']  == 'Y') | (test_annot_intuitive_df['judgment']  == 'N')]
test_annot_textual_df_clean = test_annot_textual_df[(test_annot_textual_df['judgment']  == 'Y') | (test_annot_textual_df['judgment']  == 'N')]
train_annot_intuitive_df_clean = train_annot_intuitive_df[(train_annot_intuitive_df['judgment']  == 'Y') | (train_annot_intuitive_df['judgment']  == 'N')]
train_annot_textual_df_clean = train_annot_textual_df[(train_annot_textual_df['judgment']  == 'Y') | (train_annot_textual_df['judgment']  == 'N')]

print(len(test_annot_intuitive_df_clean))
print(len(test_annot_textual_df_clean))


df = test_annot_intuitive_df_clean.merge(test_annot_textual_df_clean, on=['id','disease'])
print(df[df['judgment_x'] != df['judgment_y']])

print(len(train_annot_intuitive_df_clean))
print(len(train_annot_textual_df_clean))

df = train_annot_intuitive_df_clean.merge(train_annot_textual_df_clean, on=['id','disease'])
print(df[df['judgment_x'] != df['judgment_y']])


7385
2257
       source_x disease  id judgment_x source_y judgment_y
1712  intuitive      OA   8          N  textual          Y
8598
2676
      source_x disease    id judgment_x source_y judgment_y
571  intuitive     CHF  1072          Y  textual          N


Let's remove those two records from the textual table and recheck.

In [13]:
df = test_annot_textual_df_clean
df = df.reset_index()
index_names = df[(df['disease'] == 'OA') & (df['id'] == 8)].index
test_annot_textual_df_clean = df.drop(index_names)

df = train_annot_textual_df_clean
df = df.reset_index()
index_names = df[(df['disease'] == 'CHF') & (df['id'] == 1072)].index
train_annot_textual_df_clean = df.drop(index_names)

print(len(test_annot_textual_df_clean))
df = test_annot_intuitive_df_clean.merge(test_annot_textual_df_clean, on=['id','disease'])
print(df[df['judgment_x'] != df['judgment_y']])

print(len(train_annot_textual_df_clean))
df = train_annot_intuitive_df_clean.merge(train_annot_textual_df_clean, on=['id','disease'])
print(df[df['judgment_x'] != df['judgment_y']])


2256
Empty DataFrame
Columns: [source_x, disease, id, judgment_x, index, source_y, judgment_y]
Index: []
2675
Empty DataFrame
Columns: [source_x, disease, id, judgment_x, index, source_y, judgment_y]
Index: []


The paper does a classification model for each disase seperately.  Need to be able to loop through each. Using the seperate files  seems to come closer to the disease counts the paper discusses before pre-processing, so we will use this data. The study must have done some additional processing that is not evident from the paper, so our results may be a little different.

In [14]:
disease_list = train_annot_intuitive_df_clean['disease'].unique().tolist()

train_annot_all_df_clean = pd.concat([train_annot_intuitive_df_clean,train_annot_textual_df_clean])
train_annot_all_df_clean = train_annot_all_df_clean.drop(['source','index'], axis=1)
train_annot_all_df_clean = train_annot_all_df_clean.drop_duplicates()

test_annot_all_df_clean = pd.concat([test_annot_intuitive_df_clean,test_annot_textual_df_clean])
test_annot_all_df_clean = test_annot_all_df_clean.drop(['source','index'], axis=1)
test_annot_all_df_clean = test_annot_all_df_clean.drop_duplicates()

annot_all_df_clean = pd.concat([train_annot_all_df_clean,test_annot_all_df_clean])
annot_all_df_clean = annot_all_df_clean.drop_duplicates()
samples = 0

for disease in disease_list:
  print('Disease:',disease)
  print('Train:',sum(train_annot_intuitive_df_clean['disease'] == disease),
        sum(train_annot_textual_df_clean['disease'] == disease),
        sum(train_annot_all_df_clean['disease'] == disease))
  print('Test:',sum(test_annot_intuitive_df_clean['disease'] == disease),
        sum(test_annot_textual_df_clean['disease'] == disease),
        sum(test_annot_all_df_clean['disease'] == disease))  
  print('All:',sum(annot_all_df_clean['disease'] == disease))
  samples = sum(annot_all_df_clean['disease'] == disease) + samples

print("Samples:",samples)


Disease: Asthma
Train: 572 76 581
Test: 471 70 476
All: 1057
Disease: CAD
Train: 548 349 568
Test: 457 299 476
All: 1044
Disease: CHF
Train: 243 245 269
Test: 434 216 454
All: 723
Disease: Depression
Train: 582 86 584
Test: 477 72 484
All: 1068
Disease: Diabetes
Train: 567 411 579
Test: 479 350 491
All: 1070
Disease: Gallstones
Train: 593 94 599
Test: 491 89 498
All: 1097
Disease: GERD
Train: 487 99 495
Test: 424 70 429
All: 924
Disease: Gout
Train: 596 71 600
Test: 500 52 502
All: 1102
Disease: Hypercholesterolemia
Train: 502 255 521
Test: 431 219 440
All: 961
Disease: Hypertension
Train: 531 452 564
Test: 446 380 473
All: 1037
Disease: Hypertriglyceridemia
Train: 587 15 591
Test: 486 10 488
All: 1079
Disease: OA
Train: 565 89 578
Test: 458 85 469
All: 1047
Disease: Obesity
Train: 553 248 571
Test: 447 201 466
All: 1037
Disease: OSA
Train: 590 88 596
Test: 493 69 498
All: 1094
Disease: PVD
Train: 556 83 561
Test: 464 64 469
All: 1030
Disease: Venous Insufficiency
Train: 526 14 526
Tes

Datasets to use for rest of the study:
* test_df [id,text] (document, clinical notes)
* train_df [id,text] (document, clinical notes)
* test_annot_all_df_clean [disease,id,judment,index] (disease, document, judgment)
* train_annot_all_df_clean [disease,id,judment,index] (disease, document, judgment)

For the annotations, we should convert judgement to a numeric label.

Our next step is to continue the preprocessing of the data.  We want to do this seperately from the annotations, they can be joined when doing classification by each disase. This includes:

* Lower-casing of the text
* Removing punctuation and numeric values from the text
* Tokenization of text 
* Lemmatizattion of the tokens
* TF-IDF matrix generation (TF-IDF Vectorizer4 from the scikit-learn library)

We have an optional parameter to remove stop words as the paper discusses the fact that stop words should be included for deep learning models.

In [15]:
#Convert Y/N to True/False
test_annot_all_df_clean['judgment'] = test_annot_all_df_clean['judgment'] == 'Y'
train_annot_all_df_clean['judgment'] = train_annot_all_df_clean['judgment'] == 'Y'

In [19]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier

import re
import string
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

cachedStopWords = stopwords.words("english")
vectorizer = TfidfVectorizer(stop_words = cachedStopWords)

In [25]:
def preparetext(df, removestopwords = False):
    wn = WordNetLemmatizer()

    ndf = df.copy()

    ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
    ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')
    ndf["lower_text"] = ndf['no_numerics_text'].apply(str.lower)

    #this has a side effect of getting rid of all of the carriage returns, etc.
    if removestopwords:
         ndf['lower_text'] =  ndf['lower_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (cachedStopWords)]))
    else:
         ndf['lower_text'] =  ndf['lower_text'].apply(lambda x: ' '.join([word for word in x.split()]))

    ndf["tokenized_text"] = ndf.apply(lambda row: word_tokenize(row['lower_text']), axis=1)

    ndf["tok_lem_text"] = ndf['tokenized_text'].apply(
        lambda lst:[wn.lemmatize(word) for word in lst])
    
    X = vectorizer.fit_transform(ndf['lower_text'])
    #print(X[0])
    print(X.shape) # For DL models needs to be dimension {n x 600}, with n the number of text documents (clinical records)
    
    clf = ExtraTreesClassifier(n_estimators=100, random_state=0)

    return ndf

In [26]:
train_df= preparetext(train_df, removestopwords=False)
#print(X[0])

C:\Users\mclop\AppData\Local\Temp\ipykernel_3268\3260479370.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
C:\Users\mclop\AppData\Local\Temp\ipykernel_3268\3260479370.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')


(611, 22099)


Create seperate dataframes with stop words included and removed.

In [58]:
train_df = preparetext(train_df, removestopwords=False)
print(train_df['tok_lem_text'][0])
train_df_ns = preparetext(train_df, removestopwords=True)
print(train_df_ns['tok_lem_text'][0])
test_df = preparetext(test_df, removestopwords=False)
print(test_df['tok_lem_text'][0])
test_df_ns = preparetext(test_df, removestopwords=True)
print(test_df_ns['tok_lem_text'][0])

C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')


  (0, 9127)	0.013080707666696108
  (0, 5112)	0.014658454513679682
  (0, 10504)	0.04376541572502435
  (0, 8188)	0.04376541572502435
  (0, 10106)	0.03780106126272796
  (0, 1768)	0.04376541572502435
  (0, 5111)	0.006551921693266091
  (0, 19462)	0.01669425586982241
  (0, 4307)	0.02513466895184306
  (0, 5406)	0.014924174592151633
  (0, 6382)	0.01504146133309574
  (0, 11632)	0.04376541572502435
  (0, 17180)	0.03661428685477051
  (0, 14793)	0.015705398194379196
  (0, 15389)	0.0131688702516156
  (0, 11408)	0.04376541572502435
  (0, 17846)	0.03661428685477051
  (0, 15598)	0.029111221572601043
  (0, 8343)	0.025924635655468882
  (0, 7399)	0.011599089024242091
  (0, 3603)	0.024112975889890153
  (0, 13239)	0.04376541572502435
  (0, 11680)	0.082252293661275
  (0, 14790)	0.04376541572502435
  (0, 17886)	0.04376541572502435
  :	:
  (0, 20785)	0.05784860687947873
  (0, 12383)	0.0357214412526387
  (0, 5057)	0.040070778093886504
  (0, 5065)	0.01695362237237521
  (0, 17608)	0.038301876381382056
  (0, 2072

C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')


  (0, 9127)	0.013080707666696108
  (0, 5112)	0.014658454513679682
  (0, 10504)	0.04376541572502435
  (0, 8188)	0.04376541572502435
  (0, 10106)	0.03780106126272796
  (0, 1768)	0.04376541572502435
  (0, 5111)	0.006551921693266091
  (0, 19462)	0.01669425586982241
  (0, 4307)	0.02513466895184306
  (0, 5406)	0.014924174592151633
  (0, 6382)	0.01504146133309574
  (0, 11632)	0.04376541572502435
  (0, 17180)	0.03661428685477051
  (0, 14793)	0.015705398194379196
  (0, 15389)	0.0131688702516156
  (0, 11408)	0.04376541572502435
  (0, 17846)	0.03661428685477051
  (0, 15598)	0.029111221572601043
  (0, 8343)	0.025924635655468882
  (0, 7399)	0.011599089024242091
  (0, 3603)	0.024112975889890153
  (0, 13239)	0.04376541572502435
  (0, 11680)	0.082252293661275
  (0, 14790)	0.04376541572502435
  (0, 17886)	0.04376541572502435
  :	:
  (0, 20785)	0.05784860687947873
  (0, 12383)	0.0357214412526387
  (0, 5057)	0.040070778093886504
  (0, 5065)	0.01695362237237521
  (0, 17608)	0.038301876381382056
  (0, 2072

C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')


  (0, 8233)	0.010998769562908581
  (0, 4605)	0.012106948215596604
  (0, 12500)	0.032923560997918336
  (0, 1523)	0.03682844514041578
  (0, 2029)	0.028355139411874956
  (0, 4321)	0.03682844514041578
  (0, 4604)	0.005644657596120385
  (0, 15883)	0.03682844514041578
  (0, 14396)	0.031666469085168264
  (0, 19363)	0.029018676855420894
  (0, 4550)	0.02977093356540834
  (0, 10149)	0.03682844514041578
  (0, 1915)	0.03682844514041578
  (0, 9606)	0.032923560997918336
  (0, 6968)	0.034544234347394086
  (0, 2679)	0.02154679446110644
  (0, 17519)	0.014514978432144254
  (0, 5537)	0.01836646353040265
  (0, 4905)	0.012357369753239861
  (0, 5788)	0.012394070580028971
  (0, 13261)	0.02259960888742331
  (0, 7888)	0.0068685130880557785
  (0, 12134)	0.012657897375603981
  (0, 17524)	0.027224649613242183
  (0, 12144)	0.019812709417474685
  :	:
  (0, 12951)	0.21962478268944058
  (0, 8313)	0.016651766993250244
  (0, 13822)	0.008063810337480654
  (0, 7824)	0.050367755885946434
  (0, 6108)	0.04088567606554019
  

C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_punc_text"] = ndf['text'].str.replace('[^\w\s]', '')
C:\Users\mclop\AppData\Local\Temp\ipykernel_8012\3740084864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf["no_numerics_text"] = ndf['no_punc_text'].str.replace('\d+', '')


  (0, 8233)	0.010998769562908581
  (0, 4605)	0.012106948215596604
  (0, 12500)	0.032923560997918336
  (0, 1523)	0.03682844514041578
  (0, 2029)	0.028355139411874956
  (0, 4321)	0.03682844514041578
  (0, 4604)	0.005644657596120385
  (0, 15883)	0.03682844514041578
  (0, 14396)	0.031666469085168264
  (0, 19363)	0.029018676855420894
  (0, 4550)	0.02977093356540834
  (0, 10149)	0.03682844514041578
  (0, 1915)	0.03682844514041578
  (0, 9606)	0.032923560997918336
  (0, 6968)	0.034544234347394086
  (0, 2679)	0.02154679446110644
  (0, 17519)	0.014514978432144254
  (0, 5537)	0.01836646353040265
  (0, 4905)	0.012357369753239861
  (0, 5788)	0.012394070580028971
  (0, 13261)	0.02259960888742331
  (0, 7888)	0.0068685130880557785
  (0, 12134)	0.012657897375603981
  (0, 17524)	0.027224649613242183
  (0, 12144)	0.019812709417474685
  :	:
  (0, 12951)	0.21962478268944058
  (0, 8313)	0.016651766993250244
  (0, 13822)	0.008063810337480654
  (0, 7824)	0.050367755885946434
  (0, 6108)	0.04088567606554019
  

Save each of these data frames for use in future notebooks.

In [30]:
test_df.to_pickle(DATA_PATH + '/test_df.pkl')
train_df.to_pickle(DATA_PATH + '/train_df.pkl')
test_df_ns.to_pickle(DATA_PATH + '/test_df_ns.pkl') 
train_df_ns.to_pickle(DATA_PATH + '/train_df_ns.pkl')
test_annot_all_df_clean.to_pickle(DATA_PATH + '/test_annot_all_df_clean.pkl') 
train_annot_all_df_clean.to_pickle(DATA_PATH + '/train_annot_all_df_clean.pkl') 